<a href="https://colab.research.google.com/github/TheBigAC/ImgToImgGAN/blob/main/GANPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Import all modules
import os
import pickle
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from PIL import Image
from keras.models import Model, Sequential
from keras.layers import Dense, ConvLSTM2D, Conv2D, Input
from keras.layers import Activation, MaxPooling2D, UpSampling2D, Dropout, Flatten, Reshape
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.preprocessing.image import load_img, array_to_img
from keras.callbacks import Callback
from google.colab import drive

In [ ]:
# @title Load and Preprocess Data
drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/AdvayChandra/MainModel/Data/"
directory = "CMPFacades/" # @param {type:"string"}
train_file_name = "traindata.pkl" # @param {type:"string"}
test_file_name = "testdata.pkl" # @param {type:"string"}

with open(path + directory + train_file_name, 'rb') as file:
  train = pickle.load(file)
  y_train = tf.data.Dataset.from_tensor_slices(train["X_train"]).batch(16)
  X_train = tf.data.Dataset.from_tensor_slices(train["y_train"]).batch(16)
file.close()

with open(path + directory + test_file_name, 'rb') as file:
  test = pickle.load(file)
  y_test = tf.data.Dataset.from_tensor_slices(test["X_test"]).batch(16)
  X_test = tf.data.Dataset.from_tensor_slices(test["y_test"]).batch(16)
file.close()

Mounted at /content/gdrive


In [ ]:
# @title Create the Generator

def build_generator():
  model = Sequential()
  model.add(Conv2D(filters=64, strides=2, kernel_size=(4, 4), padding='valid', activation='mish', input_shape=(256, 256, 3)))
  model.add(Conv2D(filters=64, strides=2, kernel_size=(4, 4), padding='valid', activation='mish'))
  model.add(Conv2D(filters=64, strides=2, kernel_size=(4, 4), padding='valid', activation='mish'))
  model.add(Flatten())
  model.add(Dropout(0.1))
  model.add(Dense(1350, activation='mish'))
  model.add(Dense(675, activation='mish'))
  model.add(Dropout(0.1))
  model.add(Dense(768, activation='mish'))
  model.add(Dense(768, activation='mish'))
  model.add(Dense(768, activation='mish'))
  model.add(Dense(256 * 256 * 3, activation='sigmoid'))
  model.add(Reshape((256, 256, 3)))
  return model
"""
def other_generator():
  model = Sequential()
  model.add(Conv2D(filters=64, strides=2, kernel_size=(4, 4), padding='same', activation='leaky_relu', input_shape=(256, 256, 3)))
  model.add(Conv2D(filters=64, strides=2, kernel_size=(4, 4), padding='same', activation='leaky_relu', input_shape=(256, 256, 3)))
  return model
"""
generator = build_generator()
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 127, 64)      3136      
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 64)        65600     
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 64)        65600     
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dropout (Dropout)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 1350)              77761350  
                                                                 
 dense_1 (Dense)             (None, 675)               9

In [ ]:
# @title Create the Discriminator

def build_discriminator():
  model = Sequential()
  model.add(Conv2D(filters=32, strides=2, kernel_size=(6, 6), padding='valid', activation='mish', input_shape=(256, 256, 3)))
  model.add(Conv2D(filters=32, strides=2, kernel_size=(3, 3), padding='valid', activation='mish', input_shape=(256, 256, 3)))
  model.add(Conv2D(filters=32, strides=2, kernel_size=(6, 6), padding='valid', activation='mish', input_shape=(256, 256, 3)))
  model.add(Flatten())
  model.add(Dropout(0.2))
  model.add(Dense(841, activation='mish'))
  model.add(Dense(116, activation='mish'))
  model.add(Dense(116, activation='mish'))
  model.add(Dense(58, activation='mish'))
  model.add(Dense(29, activation='mish'))
  model.add(Dense(29, activation='mish'))
  model.add(Dense(14, activation='mish'))
  model.add(Dense(14, activation='mish'))
  model.add(Dense(1, activation='sigmoid'))
  return model

discriminator = build_discriminator()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      3488      
                                                                 
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        9248      
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 32)        36896     
                                                                 
 flatten_1 (Flatten)         (None, 26912)             0         
                                                                 
 dropout_2 (Dropout)         (None, 26912)             0         
                                                                 
 dense_6 (Dense)             (None, 841)               22633833  
                                                                 
 dense_7 (Dense)             (None, 116)              

In [ ]:
#@title Create GAN Class

class GAN(Model):
  def __init__(self, generator, discriminator, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.generator = generator
    self.discriminator = discriminator
  def compile(self, *args, **kwargs):
    super().compile(*args, **kwargs)
    self.g_opt = Adam(learning_rate=0.0001)
    self.d_opt = Adam(learning_rate=0.00001)
  def train_step(self, x_batch_train, y_batch_train):
    real_images = y_batch_train
    fake_images = self.generator(x_batch_train, training=False)

    with tf.GradientTape() as d_tape:
      yhat_real = self.discriminator(real_images, training=True)
      yhat_fake = self.discriminator(fake_images, training=True)
      yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)

      y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)

      noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
      noise_fake = 0.15*tf.random.uniform(tf.shape(yhat_fake))
      y_realfake += tf.concat([noise_real, noise_fake], axis=0)

      total_d_loss = BinaryCrossentropy(y_realfake, yhat_realfake)

    dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables)
    self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))

    with tf.GradientTape() as g_tape:
      gen_images = self.generator(X_batch_train, training=True)
      predicted_labels = self.discriminator(gen_images, training=False)
      total_g_loss = BinaryCrossentropy(tf.zeros_like(predicted_images), predicted_images)

    ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
    self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))

    return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [ ]:
#@title Create instance of GAN
testGAN = GAN(generator, discriminator)
testGAN.compile()

In [ ]:
# @title Callback Class
class ModelMonitor(Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
      for i in range(self.num_img):
        input_image = random.choice(test["y_test"])
        generated_image = self.model.generator(input_image)
        generated_image *= 255
        generated_image.numpy()
        final_image = np.concatenate((input_image, generated_image), axis = 1)
        img = array_to_img(final_image)
        img.save(os.path.join('Images', f'generated_img_{epoch}_{i}.png'))

In [ ]:
history = testGAN.fit(X_train, y_train, epochs=5, callbacks=[ModelMonitor()])

ValueError: `y` argument is not supported when using dataset as input.